Nom Etudiant 1: 28602815 Philippe TAN
<br>
Nom Etudiant 2: 28714617 Lasha GOGRITCHIANI

# TME 2 : Median String


## Partie A : Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME precedent, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faire une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [1]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
n=3 #nb de sequences
t=10 #longuer des sequence

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    random.seed(None)
    for i in range (0, n) :
        sequence += random.choice(nuc)
    
    if upper :
        sequence.lower()
        
    return sequence

def modifMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    motifM = [nt.upper() for nt in motifM]
    
    randpos = 0
    randpositions = set()
    mutation = ""
    
    i = 0
    while (i < nbpos):
        randpos = random.randint(0, len(motifM) - 1)
        if randpos not in randpositions :
            while (mutation == motifM[randpos] or mutation == ""):
                mutation = random.choice(nuc)
            motifM[randpos] = mutation
            randpositions.add(randpos)
            mutation = ""
            i += 1
    if upper :
        motifM = [nt.lower() for nt in motifM]
        
    return "".join(motifM)

#tester modifMotif
print (modifMotif("acg", 2))


def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : longueur des séquences
    entrée n : nb de séquences à modifier
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    #Génère les séquences où implanter les motifs variables
    sequences = [generateRandomSequence(t - k) for i in range(n)]
    #Création du motif de base
    motif = generateRandomSequence(k)
    for i in range(n) :
        #Variation du motif de v nucléotides
        motifM = modifMotif(motif, v)
        #Position aléatoire dans les séquences où implanter le motif modifié
        pos = random.randint(0, t - k)
        sequences[i] = insertMotif(sequences[i], motifM, pos)
    
    return sequences

adn = implantMotifVar(k, v, t, n)
print (adn)


CTG
['TCAGTATCAC', 'GTAATCAGCC', 'ATTGGTTCAC']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [89]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

adn = implantMotifVar(k, v, t, n)
print (adn)

['CACCGTATTT', 'TCAGGGTATA', 'GTACGGTAGA', 'GTCGGTATAC', 'TAAGGGTAAG', 'TCAGGGTGTG', 'TAAGATATGC', 'AGGATATGAC', 'ACCGTATCTG', 'AACAGTAGGT']


3\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les jeux de données gnerer precedement. Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles. Pensez à eliminer tous les motifs ayant une formation diferents de motifs attendues, par exemple, les motifs ayant deux base repeté comme ACACACAC, etc.  

In [44]:
from itertools import product

def isLowComplexe(motif, m) :
    """
    Vérifie si le motif doit être éliminé en raison de sa simplicité,
    c'est-à-dire s'il a au moins m fois la même lettre ou s'il a deux lettres
    consécutives répétées.
    entrée : motif
    sortie : True si le motif doit être éliminé, False sinon
    """   
    
    if len(set(motif)) == 1 and len(motif) >= m: # si le motif contient une lettre m fois
        return True

    if len(set(motif)) >= 2 and len(motif) >= m :
        nuc = {'A':0, 'C':0, 'T':0, 'G':0}
        pairs = set()
        impairs = set()
        current = ""
        cpt = 0
        
        for i, letter in enumerate(motif) : #enumerate pour récupérer le nucléotide et son index
            nuc[letter] += 1
            
            if i%2 == 0 : #si lettre en position paire, on ajoute dans l'ensemble "pairs"
                pairs.add(motif[i])
            else : #sinon "impairs"
                impairs.add(motif[i])
                
        nbvalues = nuc.values()
        for i in nbvalues : #si on a au moins m occurrence d'un des nucléotides, alors le motif est SIMPLE
            if i >= m :
                return True
        if len(pairs) == 1 and len(impairs) == 1 : #si on a un pattern de deux lettres qui se répètent : SIMPLE
            return True
    
    return False


def removeLowComplexe(motifs, m):
    """
    Enlève les motifs peu complexes ayant 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie motifsClean: dictionnaire de motifs sans les motifs peu complexe.
    """
    
    motifsClean = []
    for motif in motifs :
        if isLowComplexe(motif, m) : # on vérifie si le motif est lowComplexe, si oui on passe au suivant
            continue 
        else : # sinon on l'ajoute dans la nouvelle dicionnaire
            motifsClean.append(motif)
        
    return motifsClean


def kmerList(kmers) :
    
    sequences = []
    for i in kmers:
        sequences.append("".join(i))
    return sequences
    
                
#Genere tous les K-mers de taille K ayant de AAA... à TTT...
allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)


print (len(kmers))
kmersValid = removeLowComplexe(kmers, 4)
print (len(kmersValid))
print (kmersValid[0])



65536
36120
AAACCCGG


In [110]:
def hamdist(str1:str, str2:str):
    """
    Calcul la distance de hamming entre deux chaînes de caractères
    entrée str1: chaîne de caractères
    entrée str2: chaîne de caractères
    sortie distance: distance de hamming
    """
    diffs = 0
    
    if len(str1) != len(str2) :
        raise ValueError("Les deux chaînes de caractères n'ont pas la même longueur")

    for i in range(len(str1)) :
        if str1[i] != str2[i] :
            diffs += 1
    
    return diffs


def totalDistance(motif:str, sequences, k):
    """
    Calcul la totalDistance
    entrée motif: sequence consensus
    entrée sequences: chaîne de caractères
    entrée k: taille du motif 
    entrée sequences: matrice de dimension txn contenant les séquences
    sortie totalDistance: somme de distance de hamming minimal
    """
    td = 0
    for seq in sequences :
        min_hamdist = 1000
        for i in range(len(seq) - k + 1) :
            subseq = seq[i : i + k]
            hammingdist = hamdist(motif, subseq)
            if hammingdist < min_hamdist :
                min_hamdist = hammingdist
        td += min_hamdist
    return td


def MedianStringSearch(allkmers, sequences, k):
    """
    Implement l'algorithme MedianStringSearch
    entrée allkmers: liste de K-mers valides
    entrée sequences: matrice de dimension txn contenant les séquences
    entrée k: taille du motif 
    sortie bestMotif: le motif que minimise les distances
    sortie bestDistance: la distance minimal
    sortie motifDist: un dictionnaire contenant les motifs et leurs distances
    """
    bestDistance = 999999
    bestMotif = ""
    motifDist = {}
    count = 0
    for kmer in allkmers:
        td = totalDistance(kmer, sequences, k)
        motifDist[kmer] = td
        if td < bestDistance :
            bestDistance = td
            bestMotif = kmer
    
    return bestMotif, bestDistance, motifDist

bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, adn, 8)
print("Meilleur motif :", bestMotif, "et meilleur distance: ", bestDistance)
#print(motifDist)
#print(len(kmersValid))
#print(len(adn))

Meilleur motif : GTGTACTC et meilleur distance:  20


In [109]:
#Apliquez l'algoritme MedianStringSearch sur les sequences du brin complementaire

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

def listComplement(seq) :
    return [reverseComplement(x) for x in seq]

bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, listComplement(adn), 8)
print("Meilleur motif :", bestMotif, "et meilleur distance: ", bestDistance)

Meilleur motif : GAGTACAC et meilleur distance:  20


## Partie B : Recherche de motifs variable sur vos données

Chercher sur le fichier "Sequence_by_Peaks_G*.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquer l'alogorithm Median Sting pour chercher les motifs dans votre fichier "Sequence_by_Peaks_G*. Il faut bien evidement enlever les motifs peu complexe.

In [112]:
import time

k=7
feq=10
top= 50
genome = "Sequence_by_Peaks_4.fasta"



def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())

    return sequence

sequences   = readFasta(genome)
#print(sequences)

In [120]:
allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)
kmersValid = removeLowComplexe(kmers, 3)

bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, sequences[0:3], k)
print("Meilleur motif :", bestMotif, "et meilleur distance: ", bestDistance)

Meilleur motif : ATGATGC et meilleur distance:  3
